In [ ]:
import pandas as pd
from data.consolidar import *
import gspread
from datetime import datetime, timedelta
from settings import get_settings

### Gerando consolidados de itens pinados

In [ ]:
inicio_periodo="1.07"
fim_periodo="12.07"


def get_df_itens(inicio_periodo,
                 fim_periodo,
                 key='1vGL01qU0IMKnHpT9swxuflPYSQ5iGKcN-2k7o_fw3rA',
                 path_client_secrets="../help_fashion_pinados/data/base/client_secrets.json",
                 path_xlsx_file = "../help_fashion_pinados/data/base/Cópia de Always  ON Fashion_ PINS FS.xlsx"
                 ):
    
    # formato de data para o datetime
    formato_data = "%d.%m"
    # iniciando gspread
    gs=gspread.oauth(path_client_secrets)
    # puxando abas do sheets e salvando nomes em uma lista 
    abas = gs.open_by_key(key).worksheets()
    abas_name = [x.title for x in abas]
    # df a ser retornado no final
    df_return = pd.DataFrame(columns=['item_id','horário','dia'])
    # formatando datas de inicio de fim
    inicio_periodo = datetime.strptime(inicio_periodo,formato_data).date()
    fim_periodo = datetime.strptime(fim_periodo,formato_data).date()

    # rodando abas na lista de abas do sheets
    for nome in abas_name:
        # checando se ela esta no periodo indicado na função
        data_aba = datetime.strptime(nome,formato_data).date()
        if data_aba>=inicio_periodo and data_aba<=fim_periodo:
            # lendo aba no xlsx 
            df=pd.read_excel(path_xlsx_file,sheet_name=nome)
            # filtrando linhas vazias
            df_filt = df[['item_id','horário']].loc[df['item_id']!=""]
            # formatando horário
            df_filt['horário']=df_filt['horário'].apply(lambda x: x.replace("h",""))
            # gerando coluna com nome da aba
            df_filt['dia']=nome
            # tirando valores NaN
            df_filt=df_filt.dropna()
            # concatenando no df final
            df_return=pd.concat([df_return,df_filt])
    # resetando index
    df_return.reset_index(inplace=True,drop=True)
    # tirando valores ciêntificos
    df_return['item_id']=df_return['item_id'].apply(int)
    return df_return

df = get_df_itens(inicio_periodo,fim_periodo)

# display(df)

In [ ]:
query='''
WITH 
-- criando table para consilidar infos das abas do sheets
sheets_pinado_table as(
        Select * from ( 
            
                VALUES
                {}

        ) as t (item_id, batch, grass_date,comparative_grass_date , key )

)

SELECT t2.grass_date                                dia_pinado
        -- , date(t1.fs_st)
        ,t1.shop_id 
        ,t1.item_id
        ,t1.main_category
        ,t1.level2_global_kpi_category
        ,t1.batch_time --
        ,MAX(case when date(t1.fs_st) = date(t2.grass_date) then t1.item_promotion_price       else 0  end)    promo_price_dia_pinado
        ,SUM(case when date(t1.fs_st) = date(t2.grass_date) then t1.gross_orders               else 0  end)    orders_dia_pinado
        ,SUM(case when date(t1.fs_st) = date(t2.grass_date) then t1.gmv                        else 0  end)    gmv_dia_pinado
        ,MAX(case when date(t1.fs_st) != date(t2.grass_date) then t1.item_promotion_price  else 0  end)        promo_price_max_acumulado
        ,SUM(case when date(t1.fs_st) != date(t2.grass_date) then t1.gross_orders          else 0  end)  / 30  ADO_acumulado
        ,SUM(case when date(t1.fs_st) != date(t2.grass_date) then t1.gmv                   else 0  end) / 30   adgmv_acumulado
     
    FROM
        staging_brbi_bdcluster.flash_sales_dataset t1

    inner join sheets_pinado_table t2 
        on  cast(t1.item_id as varchar ) = t2.item_id 
        and date(t1.fs_st) between date(comparative_grass_date) and date(grass_date)
        and cast(t1.item_id as varchar) || '-' || cast(t1.batch_time as varchar)  = key 
    WHERE 
        cluster_item = 'Fashion'
    group by 
        1, 2, 3, 4, 5, 6
'''

In [ ]:

string_final=""
for lin,_df_ in df.iterrows():
    data_formatada = datetime.strptime(_df_.dia, "%d.%m").replace(year=datetime.now().year).strftime("%Y-%m-%d")
    data_convertida = datetime.strptime(data_formatada, "%Y-%m-%d")

    data_30_dias_atras = data_convertida - timedelta(days=30)
    data_30_dias_atras_formatada = data_30_dias_atras.strftime("%Y-%m-%d")

    lista = (str(_df_.item_id),
     str(int(_df_.horário)),
     data_formatada,
     data_30_dias_atras_formatada,
     f'{_df_.item_id}-{int(_df_.horário)}')
    
    if lin==df.shape[0]-1:
        string_final=f"{string_final}\n{lista}"
    else:
        string_final=f"{string_final}\n{lista},"
with open("arquivo.txt", "w") as arquivo:
    # Escreva a string no arquivo
    arquivo.write(query.format(string_final))


In [ ]:
from pyhive import presto
from time import sleep
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

settings=get_settings()

# Ignora warnings, caso tenha algum erro irrastreavel, descomentar e rodar novamente

# Sempre imprimir todas as linhas possíveis (pode travar impressões longas)
pd.set_option('display.max_rows', None)

# Conexão - Pegar usuário no DataHub - https://datasuite.shopee.io/datahub
conn =  presto.Connection(host=settings.PRESTO_HOST, 
                        port=settings.PRESTO_PORT, 
                        username= settings.PRESTO_USERNAME, 
                        password= settings.PRESTO_PASSWORD,
                        protocol='https',
                        source=f'(25)-(brbi-adhoc)-({settings.PRESTO_USERNAME})-(jdbc)-({settings.PRESTO_USERNAME})-(SG)'
                        )

In [ ]:
df_extract = pd.read_sql(query.format(string_final) ,conn)

In [ ]:
df_extract.to_csv("base_retorno_itens_pinados.csv",index=False,encoding="utf-8-sig")